## Create Environment, and run a Randomized Test

In [1]:
import gym 
import random

In [2]:
# Using the CartPole Env
env = gym.make('CartPole-v0')

# Get the States possible in the environment
states = env.observation_space.shape[0]

# Get Possible Actions possible in the environment
actions = env.action_space.n

In [3]:
# Print Number of States and Actions
print('States=', states, 'and Actions=', actions)

States= 4 and Actions= 2


#### Run the randmised test

In [4]:
# Episodes = number of tests
episodes = 10

for episode in range(1, episodes+1):
    # Set current state of the environment
    state = env.reset()
    
    # Set terminating of environment false
    done = False
    
    # Set initial score to 0 
    score = 0 
    
    while not done:
        # Render GUI of Test
        # env.render()
        
        # Select a random action
        action = random.choice([0,1])
        
        # Execute the selected action
        n_state, reward, done, info = env.step(action)
        
        # Update score
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:11.0
Episode:2 Score:26.0
Episode:3 Score:48.0
Episode:4 Score:20.0
Episode:5 Score:31.0
Episode:6 Score:18.0
Episode:7 Score:26.0
Episode:8 Score:17.0
Episode:9 Score:41.0
Episode:10 Score:17.0


## Create Deep Learning Model 

In [ ]:
# Import Libraries
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
# Function to build the deep learning model
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
# Generate Model
model = build_model(states, actions)

In [ ]:
# Print Model Summary
model.summary()

## Build Agent using Keras-RL

In [ ]:
# Import Libraries
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
# Function to build Agent Object
def build_agent(model, actions):
    # Set Policy
    policy = BoltzmannQPolicy()
    
    # Set Memory Constraints
    memory = SequentialMemory(limit=50000, window_length=1)
    
    # Set DQN Agent
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
# Build Agent using Model, and actions
dqn = build_agent(model, actions)

# Compile Agent
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

# Train-fit Agent
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

#### Test DQN Agent

In [ ]:
# Run Test for 100 episodes
scores = dqn.test(env, nb_episodes=100, visualize=False)

# Print the scores
print(np.mean(scores.history['episode_reward']))

#### Save Agent

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

#### Reload Agent from File

In [ ]:
# Remove OLD components
del model
del dqn
del env

In [ ]:
# Init NEW components
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
# Load Weights
dqn.load_weights('dqn_weights.h5f')

In [ ]:
# Run Test for 5 episodes
_ = dqn.test(env, nb_episodes=5, visualize=True)